In [1]:
import os 
import pandas as pd
import tensorflow as tf
import numpy as np
import pydicom as dicom
import cv2

In [2]:
root_dir = "../"
root_dir = os.path.abspath(root_dir)
data_path = os.path.join(root_dir, "data")
raw_data_path = os.path.join(data_path, "raw")
processed_path = os.path.join(data_path, "processed")

DEAL WITH LABELING DATA

In [3]:
#Label file
label_file = os.path.join(raw_data_path, "labels.csv")

In [4]:
d_frame = pd.read_csv(label_file)
d_frame["label"] = (d_frame["Normal"] + (2* d_frame["Cancer"]) + (3* d_frame["Actionable"]) + (4* d_frame["Benign"])) - 1

In [5]:
d_frame

,PatientID,StudyUID,View,Normal,Actionable,Benign,Cancer,label
0,DBT-P00013,DBT-S00163,rmlo,0,0,1,0,3
1,DBT-P00023,DBT-S04378,lcc,0,1,0,0,2
2,DBT-P00023,DBT-S04378,lmlo,0,1,0,0,2
3,DBT-P00023,DBT-S04378,rcc,0,1,0,0,2
4,DBT-P00023,DBT-S04378,rmlo,0,1,0,0,2
...,...,...,...,...,...,...,...,...
19143,DBT-P04356,DBT-S00073,rmlo,1,0,0,0,0
19144,DBT-P01481,DBT-S01753,lcc,1,0,0,0,0
19145,DBT-P01481,DBT-S01753,lmlo,1,0,0,0,0
19146,DBT-P01481,DBT-S01753,rcc,1,0,0,0,0


In [6]:
np.unique(d_frame["label"].to_numpy())

array([0, 1, 2, 3], dtype=int64)

In [83]:
def get_label_info(file_path: str) -> pd.DataFrame:
    d_frame = pd.read_csv(file_path)
    d_frame["label"] = (d_frame["Normal"] + (2* d_frame["Cancer"]) 
                        + (3* d_frame["Actionable"]) + (4* d_frame["Benign"])) - 1
    
    ret_df = d_frame[["PatientID", "StudyUID", "label"]]
    ret_df = ret_df.drop_duplicates()

    return ret_df

In [84]:
label_info = get_label_info(label_file)
label_info

,PatientID,StudyUID,label
0,DBT-P00013,DBT-S00163,3
1,DBT-P00023,DBT-S04378,2
5,DBT-P00024,DBT-S03255,3
7,DBT-P00029,DBT-S00044,0
11,DBT-P00032,DBT-S04706,0
...,...,...,...
19128,DBT-P02747,DBT-S00814,0
19132,DBT-P02106,DBT-S01190,0
19136,DBT-P02569,DBT-S01802,0
19140,DBT-P04356,DBT-S00073,0


GET IMAGE DATA

In [10]:
#fEATURE DEFINITIONS

def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [69]:
def extract_file_meta_data(raw_path: str, file_name: str):
    dframe = pd.read_csv(os.path.join(raw_path, file_name))
    meta_data = dframe[["Subject ID", "Study Date", "File Location"]]
    meta_data.rename(columns={"Subject ID" :"PatientID"}, inplace=True)   

    meta_data = meta_data.copy()
    meta_data.loc[:, 'file_path'] = meta_data.apply(lambda row: os.path.join(raw_path, os.path.normpath(row['File Location']), "1-1.dcm"), axis=1)
    
    
    
    return meta_data#[["Study Date", "PatientID", "file_path"]]
    


all_file_paths = extract_file_meta_data(raw_path=os.path.join(raw_data_path, "manifest-1617905855234")
                                          , file_name="metadata.csv")

all_file_paths

C:\Users\willi\AppData\Local\Temp\ipykernel_4436\2999917423.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_data.rename(columns={"Subject ID" :"PatientID"}, inplace=True)


,PatientID,Study Date,File Location,file_path
0,DBT-P00003,01-01-2000,.\Breast-Cancer-Screening-DBT\DBT-P00003\01-01...,c:\Users\willi\Desktop\AIPortfolio\medical_ima...
1,DBT-P00003,01-01-2000,.\Breast-Cancer-Screening-DBT\DBT-P00003\01-01...,c:\Users\willi\Desktop\AIPortfolio\medical_ima...
2,DBT-P00003,01-01-2000,.\Breast-Cancer-Screening-DBT\DBT-P00003\01-01...,c:\Users\willi\Desktop\AIPortfolio\medical_ima...
3,DBT-P00003,01-01-2000,.\Breast-Cancer-Screening-DBT\DBT-P00003\01-01...,c:\Users\willi\Desktop\AIPortfolio\medical_ima...
4,DBT-P00005,01-01-2000,.\Breast-Cancer-Screening-DBT\DBT-P00005\01-01...,c:\Users\willi\Desktop\AIPortfolio\medical_ima...
...,...,...,...,...
193,DBT-P00050,01-01-2000,.\Breast-Cancer-Screening-DBT\DBT-P00050\01-01...,c:\Users\willi\Desktop\AIPortfolio\medical_ima...
194,DBT-P00050,01-01-2000,.\Breast-Cancer-Screening-DBT\DBT-P00050\01-01...,c:\Users\willi\Desktop\AIPortfolio\medical_ima...
195,DBT-P00051,01-01-2000,.\Breast-Cancer-Screening-DBT\DBT-P00051\01-01...,c:\Users\willi\Desktop\AIPortfolio\medical_ima...
196,DBT-P00051,01-01-2000,.\Breast-Cancer-Screening-DBT\DBT-P00051\01-01...,c:\Users\willi\Desktop\AIPortfolio\medical_ima...


In [108]:
def append_labe_data(input_df, label_df):
    print(input_df.shape)
    print(label_df.drop_duplicates().shape)

    input_df = input_df.reset_index()
    label_df = label_df.reset_index()

    df_result = pd.merge(input_df, label_df, on = "PatientID", how='left')

    df_result['StudyUID'] = df_result['StudyUID'].str.lower()
    df_result['File Location'] = df_result['File Location'].str.lower()
    df_result = df_result[df_result.apply(lambda row: row['StudyUID'].lower() in row['File Location'].lower() 
                                          if pd.notna(row['StudyUID']) and pd.notna(row['File Location']) else False, axis=1)]
    
    ret_df = df_result[["PatientID", "StudyUID", "label", "file_path"]]

    return ret_df


test = append_labe_data(all_file_paths, label_info)

(198, 4)
(4838, 3)
(198, 4)


In [109]:
def single_image_loader(input_path: str) -> np.array:
    output_height = 256
    output_width = 256
    ds = dicom.dcmread(input_path)
    ds[(0x0028, 0x0101)].value = 16
    pix_arr = ds.pixel_array
    pix_arr = [cv2.resize(slice_data, (output_width, output_height)) for slice_data in pix_arr]    
    np_img = np.array(pix_arr)
    np_img = np.transpose(np_img, (1, 2, 0))
    return np_img

In [110]:
def convert_to_record(image_data, label, file_name):
    rows, columns, depth = image_data.shape
    print(f"Writting")

    writer = tf.io.TFRecordWriter(file_name)
    image_raw = image_data.tobytes()

    example = tf.train.Example(features=tf.train.Features(feature={
        'height': _int64_feature(rows),
        'width': _int64_feature(columns),
        'depth': _int64_feature(depth),
        'label': _int64_feature(int(label)),
        'image_raw': _bytes_feature(image_raw)}))
    
    writer.write(example.SerializeToString())


In [114]:
for index, row in test.iterrows():
    path = row["file_path"]
    label = row["label"]

    image_data = single_image_loader(path)

    file_path = os.path.join(processed_path, f"record{index}.tfrecords")
    convert_to_record(image_data, label, file_path)

    break 
    #print(f"{path} : {label}")

Writting


In [115]:
def parse_tfrecord_fn(example_proto):
    feature_description = {
        'height': tf.io.FixedLenFeature([], tf.int64),
        'width': tf.io.FixedLenFeature([], tf.int64),
        'depth': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'image_raw': tf.io.FixedLenFeature([], tf.string),
    }

    example = tf.io.parse_single_example(example_proto, feature_description)
    
    height = tf.cast(example['height'], tf.int32)
    width = tf.cast(example['width'], tf.int32)
    depth = tf.cast(example['depth'], tf.int32)
    label = tf.cast(example['label'], tf.int32)

    image_raw = tf.io.decode_raw(example['image_raw'], tf.int16)
    image = tf.reshape(image_raw, (height, width, depth))

    return image, label

In [117]:
dataset = tf.data.TFRecordDataset([os.path.join(processed_path, "record0.tfrecords")])

# Map the parse_tfrecord_fn function to each element in the dataset
dataset = dataset.map(parse_tfrecord_fn)

# Create an iterator and get the next element
iterator = iter(dataset)
image, label = iterator.get_next()


print(label)
print(image)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(
[[[715 667 663 ... 688 679 633]
  [681 710 728 ... 642 629 602]
  [769 730 756 ... 612 605 604]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]

 [[725 691 715 ... 524 511 499]
  [718 737 747 ... 582 530 498]
  [776 806 796 ... 500 488 477]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]

 [[771 768 755 ... 530 532 534]
  [720 721 715 ... 496 503 528]
  [739 738 731 ... 484 499 512]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]

 ...

 [[  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]

 [[  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   